In [1]:
# Packages, which are not already installed in Google Colab
!pip install transformers
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 6.2 MB/s eta 0:00:0000:0100:01m
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-macosx_12_0_arm64.whl (3.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.9/288.9 kB 7.4 MB/s eta 0:00:0000:01
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 4.7 MB/s eta 0:00:00
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 6.1 MB/s eta 0:00:0000:0100:01
  Using cached frozenlist-1.3.3-cp310-cp310-macosx_11_0_arm64.whl (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.3 MB/s eta 

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import evaluate
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, get_scheduler

In [3]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive, files

  drive.mount('/mnt/')
  path = "/mnt/MyDrive/mental/"
else:
  path = "./"

model_path = path + "mental_weights.pth"

In [2]:
naming_convetion = {
    "tweet": "text",
    "target": "label",
    "post": "text",
    "class_id": "label",
}

In [4]:
def read_dataset(path):
    df = pd.read_csv(path)
    df = df.rename(columns=naming_convetion)
    required_columns = np.unique(list(naming_convetion.values()))
    print(required_columns)
    df = df[required_columns]
    x =  df.to_dict("list")
    return x

In [5]:
# Load Redit Posts
reddit_set = {}
for s in ["train", "test", "val"]:
    reddit_set[s] = Dataset.from_dict(read_dataset(path + f"/data/reddit/posts_{s}.csv"))

#reddit_set = Dataset.from_dict(reddit_set)
reddit_set = DatasetDict(reddit_set)
reddit_set

['label' 'text']
['label' 'text']
['label' 'text']


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 13727
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1488
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 1716
    })
})

In [6]:
num_labels = 6
num_labels = max(reddit_set["train"]["label"]) + 1

# Tokenize Input

In [7]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_set = reddit_set.map(tokenize_function, batched=True)

Map:   0%|          | 0/13727 [00:00<?, ? examples/s]

Map:   0%|          | 0/1488 [00:00<?, ? examples/s]

Map:   0%|          | 0/1716 [00:00<?, ? examples/s]

In [8]:
tokenized_set = tokenized_set.remove_columns(["text"])
tokenized_set = tokenized_set.rename_column("label", "labels")
tokenized_set.set_format("torch")

In [9]:
small_train_dataset = tokenized_set["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_set["test"].shuffle(seed=42).select(range(1000))

In [10]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

# Model

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=num_labels)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Train


In [16]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [20]:
num_epochs = 50
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [21]:
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/6250 [00:00<?, ?it/s]

In [22]:
metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.77}

In [24]:
torch.save(model.state_dict(), path+ "bert_fine_tuned_20epochs.pth")

## Compare with MentalBert

**TODO**


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
model_mental_bert = AutoModel.from_pretrained("mental/mental-bert-base-uncased", num_labels=num_labels)

tokenized_set = reddit_set.map(tokenize_function, batched=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

metric = evaluate.load("accuracy")
model_mental_bert.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_mental_bert(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()